## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
# from config import g_key
import sys
sys.path.append('../')
from config import g_key 

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Georgetown,MY,5.4112,100.3354,83.89,83,20,0.00,few clouds
1,1,Tuatapere,NZ,-46.1333,167.6833,40.35,94,96,2.82,overcast clouds
2,2,Katsuura,JP,35.1333,140.3000,74.84,93,100,6.78,light rain
3,3,Busselton,AU,-33.6500,115.3333,59.63,73,83,20.85,broken clouds
4,4,Rikitea,PF,-23.1203,-134.9692,65.77,88,100,6.62,moderate rain


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 70
What is the maximum temperature you would like for your trip? 80


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]

preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
2,2,Katsuura,JP,35.1333,140.3000,74.84,93,100,6.78,light rain
5,5,Dessau,DE,51.8386,12.2456,77.38,38,79,15.41,broken clouds
13,13,Kieta,PG,-6.2167,155.6333,77.79,85,80,5.59,broken clouds
18,18,Mahebourg,MU,-20.4081,57.7000,70.25,73,75,10.36,broken clouds
22,22,Khao Wong,TH,16.7001,104.0902,77.16,93,99,2.98,moderate rain
24,24,Auki,SB,-8.7676,160.7034,75.88,85,39,2.84,scattered clouds
26,26,Kapaa,US,22.0752,-159.3190,77.32,87,75,16.11,broken clouds
28,28,Jamestown,US,42.0970,-79.2353,73.94,59,20,11.50,few clouds
30,30,Ode,NG,7.7899,5.7117,73.15,89,99,6.40,overcast clouds
32,32,Albany,US,42.6001,-73.9662,79.57,60,6,6.35,clear sky


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()

City_ID                0
City                   0
Country                1
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()
clean_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
2,2,Katsuura,JP,35.1333,140.3000,74.84,93,100,6.78,light rain
5,5,Dessau,DE,51.8386,12.2456,77.38,38,79,15.41,broken clouds
13,13,Kieta,PG,-6.2167,155.6333,77.79,85,80,5.59,broken clouds
18,18,Mahebourg,MU,-20.4081,57.7000,70.25,73,75,10.36,broken clouds
22,22,Khao Wong,TH,16.7001,104.0902,77.16,93,99,2.98,moderate rain
24,24,Auki,SB,-8.7676,160.7034,75.88,85,39,2.84,scattered clouds
26,26,Kapaa,US,22.0752,-159.3190,77.32,87,75,16.11,broken clouds
28,28,Jamestown,US,42.0970,-79.2353,73.94,59,20,11.50,few clouds
30,30,Ode,NG,7.7899,5.7117,73.15,89,99,6.40,overcast clouds
32,32,Albany,US,42.6001,-73.9662,79.57,60,6,6.35,clear sky


In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Katsuura,JP,74.84,light rain,35.1333,140.3000,
5,Dessau,DE,77.38,broken clouds,51.8386,12.2456,
13,Kieta,PG,77.79,broken clouds,-6.2167,155.6333,
18,Mahebourg,MU,70.25,broken clouds,-20.4081,57.7000,
22,Khao Wong,TH,77.16,moderate rain,16.7001,104.0902,
24,Auki,SB,75.88,scattered clouds,-8.7676,160.7034,
26,Kapaa,US,77.32,broken clouds,22.0752,-159.3190,
28,Jamestown,US,73.94,few clouds,42.0970,-79.2353,
30,Ode,NG,73.15,overcast clouds,7.7899,5.7117,
32,Albany,US,79.57,clear sky,42.6001,-73.9662,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
       
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]

    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [9]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.loc[(hotel_df["Hotel Name"]!='')]
clean_hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Katsuura,JP,74.84,light rain,35.1333,140.3000,Katsuura Hotel Mikazuki
5,Dessau,DE,77.38,broken clouds,51.8386,12.2456,"Radisson Blu Fürst Leopold Hotel, Dessau"
13,Kieta,PG,77.79,broken clouds,-6.2167,155.6333,Uruna Bay Retreat
18,Mahebourg,MU,70.25,broken clouds,-20.4081,57.7000,Shandrani Beachcomber Resort & Spa
22,Khao Wong,TH,77.16,moderate rain,16.7001,104.0902,ลำน้ำยังรีสอร์ท
24,Auki,SB,75.88,scattered clouds,-8.7676,160.7034,Auki Motel
26,Kapaa,US,77.32,broken clouds,22.0752,-159.3190,Sheraton Kauai Coconut Beach Resort
28,Jamestown,US,73.94,few clouds,42.0970,-79.2353,Hampton Inn & Suites Jamestown
30,Ode,NG,73.15,overcast clouds,7.7899,5.7117,OWONIDADDY HOTEL AND SUITES
33,Hilo,US,74.62,broken clouds,19.7297,-155.0900,Hilo Hawaiian Hotel


In [10]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [15]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
<!-- <dt>Max Temp</dt><dd>{Max Temp} °F</dd> -->
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [16]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)

fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))